In [1]:
# add source to apt
# sudo apt update
# sudo apt install ni-daqmx
import time
import datetime
import pprint
import nidaqmx
import threading
import numpy as np
import nidaqmx.stream_writers

In [2]:
rate = 10
samples_per_read = rate

samples = 0
def aquire_as_thread():
    with nidaqmx.Task() as task:
        task.ai_channels.add_ai_voltage_chan("Dev1/ai0")

        task.timing.cfg_samp_clk_timing(rate, sample_mode=nidaqmx.constants.AcquisitionType.CONTINUOUS)

        def callback(task_handle, every_n_samples_event_type,
                     number_of_samples, callback_data):
            #print('Every N Samples callback invoked.')
            data = task.read(number_of_samples_per_channel=samples_per_read)
            with open('nilog.txt','a') as fi:
                fi.write(str(datetime.datetime.now())+'\n'+ ''.join([f'{s:.5f} ' for s in data]) +'\n')
            global samples
            samples +=1
            return 0

        task.register_every_n_samples_acquired_into_buffer_event(
            samples_per_read, callback)

        task.start()            
        while keep_alive:
            print(f' {samples}x{samples_per_read} datapoints', end = '\r')
            time.sleep(1)
            
keep_alive = True 
threading.Thread(target=aquire_as_thread, args=()).start()


In [7]:
# to stop reading thread:
if 0:
    keep_alive = False
    print(f' {samples}x{samples_per_read} datapoints')

In [3]:
old_v = 0 #v

In [17]:
# to set new voltage
new_v = 0.01# V

write_rate = 100
duration = 1
ramp = np.linspace(old_v, new_v, int(write_rate*duration))
with nidaqmx.Task() as task:
    channel = task.ao_channels.add_ao_voltage_chan("Dev1/ao0")
    task.timing.cfg_samp_clk_timing(rate=write_rate,
                                        #source='/cDAQ1/ao/SampleClock',
                                        #active_edge=nidaqmx.constants.Edge.RISING,
                                        #sample_mode=nidaqmx.constants.AcquisitionType.FINITE,
                                    samps_per_chan= len(ramp)
                                   )
    writer = nidaqmx.stream_writers.AnalogMultiChannelWriter(task.out_stream)


    task.write(ramp, auto_start=False)
    task.start()
    with open('nilog_ao0.txt','a') as f:
        f.write(str(datetime.datetime.now())+'\n'+ f'rate {write_rate}, data: ' + ''.join([f'{s:.5f} ' for s in ramp]) +'\n')
    task.wait_until_done()

old_v = new_v # reset        

In [48]:
print(old_v)

1


In [4]:
while True:
    # to set new voltage
    new_v = np.random.random()*2-1# V

    write_rate = 100
    duration = 1
    ramp = np.linspace(old_v, new_v, int(write_rate*duration))
    with nidaqmx.Task() as task:
        channel = task.ao_channels.add_ao_voltage_chan("Dev1/ao0")
        task.timing.cfg_samp_clk_timing(rate=write_rate,
                                            #source='/cDAQ1/ao/SampleClock',
                                            #active_edge=nidaqmx.constants.Edge.RISING,
                                            #sample_mode=nidaqmx.constants.AcquisitionType.FINITE,
                                        samps_per_chan= len(ramp)
                                       )
        writer = nidaqmx.stream_writers.AnalogMultiChannelWriter(task.out_stream)


        task.write(ramp, auto_start=False)
        task.start()
        with open('nilog_ao0.txt','a') as f:
            f.write(str(datetime.datetime.now())+'\n'+ f'rate {write_rate}, data: ' + ''.join([f'{s:.5f} ' for s in ramp]) +'\n')
        task.wait_until_done()
    old_v = new_v # reset        
    time.sleep(1)

KeyboardInterrupt: 